In [1]:
git_repo_filepath = '/filepath/to/gitrepo/episodic-memory-benchmark'

In [2]:
from pathlib import Path
from epbench.src.generation.benchmark_generation_wrapper import BenchmarkGenerationWrapper
book_parameters = {'indexing': 'default', 'nb_summaries': 0}
data_folder = Path(git_repo_filepath) / 'epbench' / 'data'
env_file = Path(git_repo_filepath) / '.env'

print("Generation with Claude -- 200 events")
prompt_parameters = {'nb_events': 200, 'name_universe': 'default', 'name_styles': 'default', 'seed': 0, 'distribution_events': {'name': 'geometric', 'param': 0.1}}
model_parameters = {'model_name': 'claude-3-5-sonnet-20240620', 'max_new_tokens': 4096, 'itermax': 10}
benchmark_claude_200 = BenchmarkGenerationWrapper(prompt_parameters, model_parameters, book_parameters, data_folder, env_file)

Generation with Claude -- 200 events
At iteration 0, 33.50% remaining with issues (67/200), for index: [11, 13, 16, 19, 20, 23, 25, 30, 33, 42, 44, 45, 47, 48, 50, 51, 56, 59, 62, 63, 67, 69, 70, 71, 79, 80, 85, 86, 88, 93, 96, 106, 109, 122, 125, 127, 128, 130, 136, 138, 143, 144, 146, 147, 148, 149, 150, 152, 155, 156, 160, 162, 163, 166, 169, 172, 175, 177, 178, 180, 181, 182, 185, 189, 193, 197, 199].
At iteration 1, 16.50% remaining with issues (33/200), for index: [11, 13, 16, 42, 44, 56, 59, 67, 79, 80, 93, 96, 106, 122, 127, 128, 130, 136, 143, 144, 146, 147, 150, 156, 160, 162, 163, 166, 169, 172, 175, 182, 193].
At iteration 2, 10.50% remaining with issues (21/200), for index: [13, 16, 42, 44, 56, 67, 79, 93, 96, 106, 143, 144, 146, 150, 156, 160, 162, 166, 169, 182, 193].
At iteration 3, 7.50% remaining with issues (15/200), for index: [16, 42, 44, 56, 67, 93, 96, 106, 143, 144, 146, 156, 160, 182, 193].
At iteration 4, 5.50% remaining with issues (11/200), for index: [16, 4

In [3]:
# Evaluation
from epbench.src.evaluation.precomputed_results import get_precomputed_results

experiments = [
    # in-context, book with 200 events
    {'book_nb_events': 200, 'answering_kind': 'prompting', 'answering_model_name': 'gpt-4o-mini-2024-07-18'},
    {'book_nb_events': 200, 'answering_kind': 'prompting', 'answering_model_name': 'gpt-4o-2024-08-06'},
    {'book_nb_events': 200, 'answering_kind': 'prompting', 'answering_model_name': 'claude-3-haiku-20240307'},
    {'book_nb_events': 200, 'answering_kind': 'prompting', 'answering_model_name': 'claude-3-5-sonnet-20240620'},
    {'book_nb_events': 200, 'answering_kind': 'prompting', 'answering_model_name': 'o1-mini'}
]

for i in range(len(experiments)):
    if not 'answering_embedding_chunk' in experiments[i]:
        experiments[i]['answering_embedding_chunk'] = 'n/a'
    experiments[i]['book_model_name'] = 'claude-3-5-sonnet-20240620'

print(f"{len(experiments)} experiments")

all_benchmarks = {'benchmark_claude_default_200': benchmark_claude_200}

df = get_precomputed_results(experiments, env_file, data_folder, all_benchmarks)
df

5 experiments
Document with 102870 tokens, answer with prompting using with gpt-4o-mini-2024-07-18
Document with 102870 tokens, answer with prompting using with gpt-4o-2024-08-06
Document with 102870 tokens, answer with prompting using with claude-3-haiku-20240307
Document with 102870 tokens, answer with prompting using with claude-3-5-sonnet-20240620
Document with 102870 tokens, answer with prompting using with o1-mini


,book_nb_events,answering_kind,answering_model_name,answering_embedding_chunk,book_model_name,evaluation_object
0,200,prompting,gpt-4o-mini-2024-07-18,n/a,claude-3-5-sonnet-20240620,<epbench.src.evaluation.evaluation_wrapper.Eva...
1,200,prompting,gpt-4o-2024-08-06,n/a,claude-3-5-sonnet-20240620,<epbench.src.evaluation.evaluation_wrapper.Eva...
2,200,prompting,claude-3-haiku-20240307,n/a,claude-3-5-sonnet-20240620,<epbench.src.evaluation.evaluation_wrapper.Eva...
3,200,prompting,claude-3-5-sonnet-20240620,n/a,claude-3-5-sonnet-20240620,<epbench.src.evaluation.evaluation_wrapper.Eva...
4,200,prompting,o1-mini,n/a,claude-3-5-sonnet-20240620,<epbench.src.evaluation.evaluation_wrapper.Eva...


In [4]:
i=1 # only considering the hallucinations for gpt-4o-2024-08-06
df_generated_evaluations = df.iloc[i]['evaluation_object'].df_generated_evaluations
df_generated_evaluations_0 = df_generated_evaluations[df_generated_evaluations['bins_items_correct_answer'] == '0']
df_generated_evaluations_0 = df_generated_evaluations_0[df_generated_evaluations_0['get'] == 'all']

df_generated_evaluations_0 = df_generated_evaluations_0[['q_idx',
       'cue', 'cue_completed', 'retrieval_type',
       'llm_answer', 'predicted_items', 'explanation', 'f1_score_lenient', 'debug_changed',
       'debug_existing_change']]

nb_questions_with_0 = len(df_generated_evaluations_0) # 150
df_generated_evaluations_0_fail = df_generated_evaluations_0[df_generated_evaluations_0['f1_score_lenient'] < 1]
nb_questions_with_0_fail = len(df_generated_evaluations_0_fail) # 24

df_generated_evaluations_0_fail = df_generated_evaluations_0_fail[['q_idx', 'cue', 'cue_completed', 'retrieval_type', 'llm_answer', 'predicted_items', 'debug_changed',
       'debug_existing_change']]

nb_questions_with_0_fail_inner = len(df_generated_evaluations_0_fail[df_generated_evaluations_0_fail['debug_existing_change'] == True]) # 17
nb_questions_with_0_fail_outer = len(df_generated_evaluations_0_fail[df_generated_evaluations_0_fail['debug_existing_change'] == False]) # 7

# elements involving entity queries for the inner questions 
import numpy as np
res = df_generated_evaluations_0_fail[df_generated_evaluations_0_fail['debug_existing_change'] == True][['cue']].value_counts()
nb_inner_with_entity_queries = np.sum([v for k,v in zip(res.index.tolist(), res.values) if 'ent' in k[0] ]) # 14

# elements involving temporal queries for the outer questions 
import numpy as np
res2 = df_generated_evaluations_0_fail[df_generated_evaluations_0_fail['debug_existing_change'] == False][['cue']].value_counts()
nb_outer_with_temporal_queries = np.sum([v for k,v in zip(res.index.tolist(), res2.values) if 't' in k[0] ]) # 14

print(f"Of the {nb_questions_with_0_fail} questions with 0 matching events, {nb_questions_with_0} ({100*nb_questions_with_0_fail/nb_questions_with_0}%) produced incorrect answers.")
print(f"The {nb_questions_with_0_fail} failed zero-event questions can be categorized into two types (see Table 11 in the appendix for details):")
print(f"Inner questions ({nb_questions_with_0_fail_inner} cases):")
print(f"- Questions constructed using elements present in the book") # by design
print(f"- Majority ({nb_inner_with_entity_queries}/{nb_questions_with_0_fail_inner}) involve entity-based queries")
print(f"Outer questions ({nb_questions_with_0_fail_outer} cases):")
print(f"- Questions using at least one element from outside the book (sampled from the unused universe)") # by design
if(nb_outer_with_temporal_queries == nb_questions_with_0_fail_outer):
       print(f"- All involve temporal elements")
print(f"Consistent cue patterns: {', '.join([k[0] for k in res2.index.tolist()])}")

Of the 24 questions with 0 matching events, 150 (16.0%) produced incorrect answers.
The 24 failed zero-event questions can be categorized into two types (see Table 11 in the appendix for details):
Inner questions (17 cases):
- Questions constructed using elements present in the book
- Majority (14/17) involve entity-based queries
Outer questions (7 cases):
- Questions using at least one element from outside the book (sampled from the unused universe)
- All involve temporal elements
Consistent cue patterns: (t, *, *, *), (t, *, *, c), (t, *, ent, *)


In [5]:
import warnings
warnings.filterwarnings('ignore')

# Small manual corrections, since the LLM-as-a-judge machinery would be more complex
df_generated_evaluations_0_fail_outer = df_generated_evaluations_0_fail[df_generated_evaluations_0_fail['debug_existing_change'] == False]
#print(df_generated_evaluations_0_fail_outer.iloc[3]['llm_answer']) # Astronomy night
df_generated_evaluations_0_fail_outer['predicted_items'].iloc[3] = ['Astronomy Night']
#print(df_generated_evaluations_0_fail_outer.iloc[4]['llm_answer']) # ['Storytelling Festival', 'Carnival', 'Murder Mystery Dinner']
df_generated_evaluations_0_fail_outer['predicted_items'].iloc[4] = ['Storytelling Festival', 'Carnival', 'Murder Mystery Dinner']

def get_used_elements(retrieval_type, benchmark_claude_200):
    if retrieval_type == "Times":
        return benchmark_claude_200.df_book_groundtruth['date'].unique()
    elif retrieval_type == "Spaces":
        return benchmark_claude_200.df_book_groundtruth['location'].unique()
    elif retrieval_type == "Entities":
        return benchmark_claude_200.df_book_groundtruth['entity'].unique()
    elif retrieval_type == "Event contents":
        return benchmark_claude_200.df_book_groundtruth['content'].unique()
    else:
        raise ValueError('unknown retrieval type')

import numpy as np
def adding_percentage_columns(df_generated_evaluations_0_fail):
    res = []
    res2 = []
    for k in range(len(df_generated_evaluations_0_fail)):
        retrieval_type = df_generated_evaluations_0_fail.iloc[k]['retrieval_type']
        res.append(np.mean([x in get_used_elements(retrieval_type, benchmark_claude_200) for x in df_generated_evaluations_0_fail.iloc[k]['predicted_items']]))
        res2.append(len([x for x in df_generated_evaluations_0_fail.iloc[k]['predicted_items']]))

    df_generated_evaluations_0_fail['percentage_in_book'] = res
    df_generated_evaluations_0_fail['len_answer'] = res2
    return df_generated_evaluations_0_fail

df_generated_evaluations_0_fail_outer = adding_percentage_columns(df_generated_evaluations_0_fail_outer)

df_generated_evaluations_0_fail_outer


,q_idx,cue,cue_completed,retrieval_type,llm_answer,predicted_items,debug_changed,debug_existing_change,percentage_in_book,len_answer
510,16,"(t, *, *, c)","({April 09, 2026}, *, *, {Charity Gala})",Spaces,The events related to the Charity Gala on Apri...,"[Lincoln Center, High Line]","{content, location, entity}",False,1.0,2
511,16,"(t, *, *, c)","({April 09, 2026}, *, *, {Chess Championship})",Spaces,The events related to the Chess Championship o...,"[Lincoln Center, High Line]",{content},False,1.0,2
561,14,"(t, *, ent, *)","({April 09, 2026}, *, {Zoe Rivera}, *)",Spaces,"The events involving Zoe Rivera on April 09, 2...","[American Museum of Natural History, High Line]","{content, location, entity}",False,1.0,2
566,15,"(t, *, ent, *)","({April 09, 2026}, *, {Zoe Rivera}, *)",Event contents,"On April 09, 2026, Zoe Rivera was involved in ...",[Astronomy Night],"{content, location, entity}",False,1.0,1
632,2,"(t, *, *, *)","({August 24, 2024}, *, *, *)",Event contents,"On August 24, 2024, three key events took plac...","[Storytelling Festival, Carnival, Murder Myste...","{date, location, entity}",False,1.0,3
637,1,"(t, *, *, *)","({August 24, 2024}, *, *, *)",Entities,"The events on August 24, 2024, involved the fo...","[Maya Smith, Julian Ross, Mila Gonzalez, Scarl...","{date, location, entity}",False,1.0,4
642,0,"(t, *, *, *)","({August 24, 2024}, *, *, *)",Spaces,"On August 24, 2024, the unique locations where...","[American Museum of Natural History, One World...","{date, location, entity}",False,1.0,3


In [6]:
# Manual exploration

#For the one related with time, always August 24, 2024, with (the following given by 3 different questions):
# - Locations: 'One World Trade Center', 'American Museum of Natural History', 'Trinity Church'
# - Entities: 'Scarlett Thomas', 'Julian Ross', 'Maya Smith', 'Mila Gonzalez',
# - Content: 'Storytelling Festival', 'Carnival', 'Murder Mystery Dinner',

# In Chapter 147 (on December 25, 2025): 'Storytelling Festival' <> 'American Museum of Natural History' (with other entities than the one predicted)
# In Chapter 120 (on November 13, 2026): 'Murder Mystery Dinner' <> 'One World Trade Center' <> 'Scarlett Thomas' 

df_gt = benchmark_claude_200.df_book_groundtruth

subset1 = df_gt[df_gt['location'] == 'Lincoln Center']
print(subset1)

subset2 = df_gt[df_gt['content'] == 'Murder Mystery Dinner']
print(subset2)

subset3 = df_gt[df_gt['entity'] == 'Zoe Rivera']
print(subset3)

subset4 = df_gt[df_gt['date'] == 'April 09, 2026']
print(subset4)

         chapter                date        location       entity  \
chapter                                                             
162          162      March 23, 2024  Lincoln Center  Bella Brown   
172          172  September 22, 2026  Lincoln Center    Noah Wood   

                 content                                      post_entities  \
chapter                                                                       
162         Fashion Show  {Matthew Wolf, Bruno Bourne, Thea Merchant, Mi...   
172      Astronomy Night  {Gianluca Nunez, Madeleine Combs, Zechariah Nunn}   

         n_date  n_location  n_entity  n_content  raw_generated_paragraph_idx  \
chapter                                                                         
162          13           2         3         17                          165   
172           8           2         3         10                          175   

         nb_paragraphs     style  idx_t  idx_s  idx_e  idx_c  
chapter           